# Links

https://www.tensorflow.org/deploy/distributed
https://github.com/hn826/distributed-tensorflow
https://medium.com/clusterone/how-to-write-distributed-tensorflow-code-with-an-example-on-tensorport-70bf3306adcb
https://www.oreilly.com/ideas/distributed-tensorflow
https://github.com/tmulc18/Distributed-TensorFlow-Guide/tree/master/Multiple-GPUs-Single-Machine

# Strategies for distributed learning
## Model Parallelism
When the model is too big to fit into memory on one machine, one can assign different parts of the graph to different machines. The parameters will live on that machine, and their training and update operations will happen there.

## Data Parallelism
The entire graph will live on one machine called the **parameter server (ps)**. If the amount of I/O becomes to large for a single parameter server it can be replicated, a copy of the entire graph can live on several parameter servers that stay in sync.

Training operations will be executed on multiple machines called **workers**. Each worker will be reading different data batches, computing gradients, and sending update to the parameter servers. Usually the workers will average there gradients and only a single update will be sent to the parameter server.

  * **Synchronous training:** The worker will synchronize there work. At any point in time, two workers have the exact same graph parameters values.
  * **Asynchronous training:** The workers will work asynchronously. At any point in time, two workers might have different graph parameters values.

TensorFlow has three types of nodes:
 * One or more **parameter servers** that host the graph
 * A **master worker** coordinates the training operations, and takes care of initializing the model, saving and restoring model checkpoints and saving summaries for TensorBoard. The master worker also takes care of fault-tolerance (if one ps or a worker crashes)
 * **workers** (including the master worker) handle compute training steps and send updates to the parameter servers
 
The reason you might want to have more than one parameter server is to handle a large volume of I/O from the workers. 

Setting up distributed TensorFlow requires the following steps:
 * Define the `tf.trainClusterSpec` and `tf.train.Server`
 * Assign the graph to the parameter servers and workers
 * Configure and launch a `tf.train.MonitoredTrainingSession`
 
A `tf.train.ClusterSpec` represents the set of processes that participate in a distributed TensorFlow computation.

Every `tf.train.Server` belongs to a particular cluster. A `tf.train.Server` instance encapsulates a set of devices and corresponds to a particular task in a named job. The server can communicate with any other server in the same cluster.

Using the `with tf.device` command, you can now assign nodes (either ops or variables) to a specific task of a specific job.

In the data parallelism framework, variable operations will be assigned to parameter servers and training operations to workers.

TensorFlow provides a convenient `tf.train.replica_device_setter` that automatically takes care of assigning operations to devices:


    with tf.device(tf.train.replica_device_setter(cluster_spec)):
        # define graph...
        # define training operations...
        
`tf.train.MonitoredTrainingSession` is the equivalent of `tf.Session` for distributed training. It takes care of setting up a master worker node, that will handle:
  * Initializing the graph
  * Create checkpoints
  * Exporting TensorBoard summaries
  * Starting / stopping the session
  
  
  
  
  
  
  